In [ ]:
# NTX Data Cleaning & SQL Analysis Script

# --- Library Setup ---
!pip install pandas sqlalchemy openpyxl pandasql

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandasql as psql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=fe97acdcdc66e2d2ca1ab1c4f6f0e91c37a5010f54a8e7775d0ad80f011d8e38
  Stored in directory: /root/.cache/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [ ]:
# --- Load Dataset ---
data_url = 'https://raw.githubusercontent.com/hafidzwibowo/ntx-de-technical-test/refs/heads/main/Soal%201%20-%20Data%20Transformation%20dan%20Analysis%20Case/ecommerce-session-bigquery.csv'
df = pd.read_csv(data_url)

In [ ]:
# --- Data Preview ---
print("=== Kolom 1-10 ===")
display(df.iloc[:, 0:10].head())

=== Kolom 1-10 ===


,fullVisitorId,channelGrouping,time,country,city,totalTransactionRevenue,transactions,timeOnSite,pageviews,sessionQualityDim
0,2515546493837534633,Organic Search,966564,Taiwan,(not set),NaN,NaN,1567.0,82.0,17.0
1,9361741997835388618,Organic Search,157377,France,not available in demo dataset,NaN,NaN,321.0,8.0,NaN
2,7313828956068851679,Referral,228279,United States,San Francisco,NaN,NaN,927.0,11.0,63.0
3,6036794406403793540,Organic Search,1615618,United States,Boulder,NaN,NaN,1616.0,13.0,38.0
4,7847280609739507227,Organic Search,37832,Canada,not available in demo dataset,NaN,NaN,1222.0,45.0,53.0


In [ ]:
print("=== Kolom 11-20 ===")
display(df.iloc[:, 10:20].head())

=== Kolom 11-20 ===


,date,visitId,type,productRefundAmount,productQuantity,productPrice,productRevenue,productSKU,v2ProductName,v2ProductCategory
0,20170721,1500643403,PAGE,NaN,NaN,0,NaN,9180781,Suitcase Organizer Cubes,(not set)
1,20170413,1492114470,PAGE,NaN,NaN,0,NaN,9180793,26 oz Double Wall Insulated Bottle,(not set)
2,20170727,1501198444,PAGE,NaN,NaN,0,NaN,9180838,Metal Texture Roller Pen,(not set)
3,20170731,1501522359,PAGE,NaN,NaN,0,NaN,9180838,Metal Texture Roller Pen,(not set)
4,20170728,1501300311,PAGE,NaN,NaN,0,NaN,9180838,Metal Texture Roller Pen,(not set)


In [ ]:
print("=== Kolom 21-32 ===")
display(df.iloc[:, 20:32].head())

=== Kolom 21-32 ===


,productVariant,currencyCode,itemQuantity,itemRevenue,transactionRevenue,transactionId,pageTitle,searchKeyword,pagePathLevel1,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option
0,(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
1,(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
2,(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
3,(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
4,(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN


In [ ]:
# --- Data Type Conversion ---
# Tujuan: memastikan semua tipe data sesuai untuk analisis dan transformasi.
str_cols = ['fullVisitorId', 'channelGrouping', 'country', 'city', 'date', 'type', 'productSKU', 'v2ProductName',
            'v2ProductCategory', 'productVariant', 'currencyCode', 'transactionId', 'pageTitle',
            'searchKeyword', 'pagePathLevel1', 'eCommerceAction_type', 'eCommerceAction_option']
float_cols = ['time', 'totalTransactionRevenue', 'transactions', 'timeOnSite', 'pageviews', 'sessionQualityDim',
              'visitId', 'productRefundAmount', 'productQuantity', 'productPrice', 'productRevenue',
              'itemQuantity', 'itemRevenue', 'transactionRevenue', 'eCommerceAction_step']

for col in str_cols:
    if col in df.columns:
        df[col] = df[col].astype(str)

for col in float_cols:
    if col in df.columns:
        df[col] = df[col].astype(float)

In [ ]:
# --- Isi Nilai Kosong dengan 0 (Untuk Kolom Numerik Tertentu) ---
# Alasan: logika bisnis menganggap nilai kosong = tidak terjadi (transaksi, revenue, dll)
fill_zero_cols = [
    'totalTransactionRevenue', 'transactions', 'timeOnSite', 'pageviews', 'sessionQualityDim',
    'productRefundAmount', 'productQuantity', 'itemQuantity', 'itemRevenue', 'productRevenue',
    'transactionRevenue'
]

for col in fill_zero_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)

In [ ]:
# --- Isi Nilai Kosong pada Kolom Kategorikal dengan NaN ---
# Alasan: beberapa kolom lebih bermakna jika dibiarkan NaN daripada 0 atau string kosong
fill_na_cols = {
    'channelGrouping': np.nan,
    'country': np.nan,
    'city': np.nan,
    'type': np.nan,
    'productSKU': np.nan,
    'v2ProductName': np.nan,
    'v2ProductCategory': np.nan,
    'productVariant': np.nan,
    'currencyCode': np.nan,
    'transactionId': np.nan,
    'pageTitle': np.nan,
    'searchKeyword': np.nan,
    'pagePathLevel1': np.nan,
    'eCommerceAction_type': np.nan,
    'eCommerceAction_option': np.nan
}

for col, val in fill_na_cols.items():
    if col in df.columns:
        df[col] = df[col].fillna(val)

In [ ]:
# --- Filter Baris dengan Harga Produk 0 ---
# Alasan: dianggap tidak relevan atau anomali untuk analisis transaksi
df = df[df['productPrice'] != 0]

In [ ]:
# --- Normalisasi Nama Negara ---
df['country'] = df['country'].replace({
    '(not set)': 'Unknown',
    'U.S. Virgin Islands': 'United States',
    'Aruba': 'United States',
    'Macedonia (FYROM)': 'North Macedonia',
    'Bosnia & Herzegovina': 'Bosnia and Herzegovina',
    'Cura\u00e7ao': 'Netherlands'
})


<ipython-input-10-6eb464fd2fd3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['country'] = df['country'].replace({


In [ ]:
# --- Normalisasi Nama Produk ---
df['v2ProductName'] = df['v2ProductName'].replace({
    '7&quot; Dog Frisbee': '7 Dog Frisbee',
    '7" Dog Frisbee': '7 Dog Frisbee'
})

<ipython-input-11-8ba12017b9ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['v2ProductName'] = df['v2ProductName'].replace({


In [ ]:
# --- SQL Query Setup ---
pysqldf = lambda q: psql.sqldf(q, globals())

In [ ]:
# === CASE 1 ===
# Objective: Top 5 channel grouping by revenue for the top country
query_case1 = """
WITH top_country AS (
    SELECT country, SUM(totalTransactionRevenue) AS total_revenue
    FROM df
    GROUP BY country
    ORDER BY total_revenue DESC
    LIMIT 1
)
SELECT
    channelGrouping,
    df.country,
    SUM(totalTransactionRevenue) AS total_revenue
FROM df
JOIN top_country ON df.country = top_country.country
GROUP BY channelGrouping, df.country
ORDER BY total_revenue DESC
LIMIT 5;
"""

result_case1 = pysqldf(query_case1)
result_case1['total_revenue'] = result_case1['total_revenue'].apply(lambda x: f"{x:,.0f}")
print(result_case1)

  channelGrouping        country   total_revenue
0        Referral  United States  55,284,270,000
1  Organic Search  United States  27,211,520,000
2          Direct  United States  13,878,200,000
3     Paid Search  United States   4,190,910,000
4         Display  United States   1,665,900,000


In [ ]:
# === CASE 2 ===
# Objective: Identify users who spent a long time on site but viewed few pages
# Step: Replace 0 with mean (non-zero only) to avoid skew
for col in ['timeOnSite', 'pageviews', 'sessionQualityDim']:
    nonzero_mean = df[df[col] != 0][col].mean()
    df[col] = df[col].replace(0, nonzero_mean)

query_case2 = """
WITH user_avg AS (
    SELECT
        fullVisitorId,
        AVG(timeOnSite) AS avg_time,
        AVG(pageviews) AS avg_views,
        AVG(sessionQualityDim) AS avg_quality
    FROM df
    GROUP BY fullVisitorId
),
overall_avg AS (
    SELECT
        AVG(timeOnSite) AS mean_time,
        AVG(pageviews) AS mean_views
    FROM df
)
SELECT
    ua.fullVisitorId,
    ua.avg_time,
    ua.avg_views,
    ua.avg_quality
FROM user_avg ua
CROSS JOIN overall_avg oa
WHERE ua.avg_time > oa.mean_time AND ua.avg_views < oa.mean_views
ORDER BY ua.avg_time DESC;
"""

result_case2 = pysqldf(query_case2)
print(result_case2.head())


         fullVisitorId  avg_time  avg_views  avg_quality
0  1774577907793414721    5120.0       21.0         57.0
1   422830280523121022    3917.0       14.0         57.0
2  4512938772125368249    3602.0       17.0         57.0
3  4300482509299944914    3505.0       21.0         57.0
4  2105996551023435759    3010.0       21.0         57.0


In [ ]:
# === CASE 3 ===
# Objective: Top 10 products by net revenue, flag those with high refund
query_case3 = """
SELECT
    v2ProductName,
    SUM(productRevenue) AS total_revenue,
    SUM(productQuantity) AS total_qty,
    SUM(productRefundAmount) AS total_refund,
    SUM(productRevenue) - SUM(productRefundAmount) AS net_revenue,
    CASE
        WHEN SUM(productRefundAmount) > 0.1 * SUM(productRevenue) THEN 'FLAGGED'
        ELSE 'OK'
    END AS refund_flag
FROM df
WHERE v2ProductName IS NOT NULL
GROUP BY v2ProductName
ORDER BY net_revenue DESC
LIMIT 10;
"""

result_case3 = pysqldf(query_case3)
print(result_case3)


                               v2ProductName  total_revenue  total_qty  \
0  YouTube Men's Short Sleeve Hero Tee Black            0.0        0.0   
1                        YouTube Luggage Tag            0.0        0.0   
2                      YouTube Custom Decals            0.0        0.0   
3                                   Yoga Mat            0.0        0.0   
4                             Windup Android            0.0        0.0   
5                        Waterproof Gear Bag            0.0        2.0   
6                         Waterpoof Gear Bag            0.0        1.0   
7               Switch Tone Color Crayon Pen            0.0        0.0   
8                   Suitcase Organizer Cubes            0.0        0.0   
9                            Straw Beach Mat            0.0        5.0   

   total_refund  net_revenue refund_flag  
0           0.0          0.0          OK  
1           0.0          0.0          OK  
2           0.0          0.0          OK  
3           0